In [1]:
import math
import numpy as np
import random

In [4]:
class Value:

    def __init__(self, data, _children = (), _op = '', label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self.label = label
        self._op = _op


    def __repr__(self):
        return f"Value(data = {self.data}"


    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _children = (self, other), _op = '+')

        def _add_backward():
            self.grad += out.grad * 1.0
            other.grad += out.grad * 1.0
        out._backward = _add_backward

        return out


    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _children =(self, other), _op = '*')
        
        def _mul_backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _mul_backward
        
        return out


    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _pow_backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _pow_backward

    def __rmul__(self, other): # other * self
        return self*other
    
    def __truediv__(self, other): # self / other
        return self * other**-1
    
    def __neg__(self): # -self
        return self * -1
    
    def __sub__(self, other): # when - sign is detected
        return self + (-other)

    def __rsub__(self,other):
        return self + (-other)

    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t, _children = (self,), _op = 'tanh')

        def _tan_backward():
            self.grad += 1-t**2
        out._backward = _tan_backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
    
        def _exp_backward():
            self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
        out._backward = _exp_backward
        
        return out

    def backward_pass(self):

        topo = []
        visited = set()

        def build_topo(x):
            if x not in visited:
                visited.add(x)
                for i in x._prev:
                    build_topo(i)
                topo.append(x)
        build_topo(self)
        
        self.grad = 1
        for node in reversed(topo):
            node._backward()

In [5]:
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')
b = Value(6.8813735870195432, label = 'b')

w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.data

0.7071067811865476

In [26]:
o.backward_pass()
x2.grad

0.4999999999999999

In [6]:
o.grad = 1.0
o._backward()
n.grad

n._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()

In [7]:
class Neuron:
    
    def __init__(self, n_inputs):
        self.w = [Value(random.uniform(-1,1)) for i in range(n_inputs)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        # wi * xi + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]


x = [1.0, 2.0, 3.0]
h1 = Neuron(len(x))
h1(x)

Value(data = 0.994727402112036

In [8]:
class Layer:
    
    def __init__(self, n_in, n_out):
        self.neurons = [Neuron(n_in) for i in range(n_out)]
    
    def __call__(self, x):
        outs = [neuron(x) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]


x = [1.0, 2.0, 3.0]
hidden_layer_1 = Layer(len(x), 4)
hidden_layer_1(x)

[Value(data = -0.34832816936502065,
 Value(data = 0.8557812034836113,
 Value(data = 0.40281254497670105,
 Value(data = 0.819303001716833]

In [9]:
class MLP:
    
    # n_in = len(x) = 3 ; n_outs = [4, 4, 1]
    def __init__(self, n_in, n_outs):
        sz = [n_in] + n_outs # [3, 4, 4, 1]
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(n_outs))]
    
    def __call__(self, x):
        for layer in self.layers:
            out = layer(x)
        return out
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


x = [1.0, 2.0, 3.0]
mlp = MLP(len(x), [4,4,1])
mlp(x)

Value(data = 0.9996476266996333

In [52]:
class new_MLP:
    # n_in = len(x) = 3 ; n_outs = [4, 4, 1]
    def __init__(self, n_in, n_outs):
        sz = [n_in] + n_outs # [3, 4, 4, 1]
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(n_outs))]
    def __call__(self, x):
        #self.layers[0](x), self.layers[1](self.layers[0](x)), self.layers[2](self.layers[1](self.layers[0](x)))
            

        #self.layers = a
        #x0 = a[0](x)
        #x1 = a[1](x0)
        #x2 = a[2](x1)
        xs = [x, ]
        for i in range(len(self.layers)):
            new_x = self.layers[i](xs[i])
            xs.append(new_x)
        return xs[-1]

            # new_x = a[0](x) ; new_x = a[1](new_x)

x = [1.0, 2.0, 3.0]       
new_mlp = new_MLP(len(x), [4,4,1])

In [15]:
new_mlp(x)

[Value(data = 0.4835167848602081]

In [10]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets
y_pred = [mlp(x) for x in xs]
y_pred

[Value(data = 0.9979451934645566,
 Value(data = 0.989161378400846,
 Value(data = 0.9614474778854962,
 Value(data = 0.8304831901742407]

In [18]:
for y_t,y_p in zip(ys, y_pred):
    l = y_p - y_t
    print(l)

Value(data = -0.002054806535443432
Value(data = 1.989161378400846
Value(data = 1.9614474778854962
Value(data = -0.16951680982575934


In [19]:
loss = sum((y_out - y_true)**2 for y_out, y_true in zip(y_pred, ys))
loss

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [33]:
ypred = [mlp(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'